# Definição de funções e imports necessários

In [1]:
import pandas as pd
import numpy as np
import json
import excel2json
from geopy.geocoders import Nominatim
import pymongo

### Funções:

In [15]:
def readCSV (path):
    return pd.read_csv(path, sep = ";");

def openDataBaseMongoDB ():
    client = pymongo.MongoClient("mongodb://<ADMIN>:<PASSWORD>@criedata-shard-00-00.urzuu.mongodb.net:27017,criedata-shard-00-01.urzuu.mongodb.net:27017,criedata-shard-00-02.urzuu.mongodb.net:27017/<dbname>?ssl=true&replicaSet=atlas-6db9id-shard-0&authSource=admin&retryWrites=true&w=majority");
    db = client.schoolData;
    return db.dataINEPSchools;

def dataframeToJSONConverter (dataframe):
    dataJSON = dataframe.to_json(orient="records")
    return json.loads(dataJSON)

### Conexão com o BD:

In [3]:
collection = openDataBaseMongoDB()

### Extração dos dados

In [4]:
infoSchools = readCSV(r'C:\Users\mbrug\Desktop\schoolDataCollectionPelotas\Datasets\INEP\InformacoesGeraisDasEscolas.csv');

In [5]:
infoSchools.head()

,Restrição de Atendimento,Escola,Código INEP,UF,Município,Localização,Localidade Diferenciada,Categoria Administrativa,Endereço,Telefone,Dependência Administrativa,Categoria Escola Privada,Conveniada Poder Público,Regulamentação pelo Conselho de Educação,Porte da Escola,Etapas e Modalidade de Ensino Oferecidas,Outras Ofertas Educacionais,Latitude,Longitude
0,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,COLEGIO TIRADENTES PELOTAS,43000304,RS,Pelotas,Urbana,A escola não está em área de localização difer...,Pública,"AV DUQUE DE CAXIAS 546, 546 FRAGATA. 96030-003...",(53) 32812866,Estadual,Não Informado,Não,Sim,Entre 201 e 500 matrículas de escolarização,Ensino Médio,NaN,-31.75701,-52.37308
1,ESCOLA PARALISADA,EEI NOSSOS AMIGUINHOS,43001084,RS,Pelotas,Urbana,Não Informado,Privada,"RUA VOLUNTARIOS DA PATRIA, 1927. CENTRO. 96015...",(53) 30256230,Privada,Não Informado,Não,NaN,NaN,NaN,NaN,NaN,NaN
2,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,EEI CRIANCA AMADA,43001165,RS,Pelotas,Urbana,A escola não está em área de localização difer...,Privada,"AVENIDA DOMINGOS JOSE DE ALMEIDA, 1077 AREAL. ...",(53) 30282009,Privada,Particular,Não,Sim,Entre 51 e 200 matrículas de escolarização,Educação Infantil,NaN,NaN,NaN
3,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,EEI MUNDO DO MICKEY,43001173,RS,Pelotas,Urbana,A escola não está em área de localização difer...,Privada,"RUA BALDOMERO TRAPAGA, 132 PORTO. 96075-540 Pe...",(53) 32279908,Privada,Particular,Não,Sim,Até 50 matrículas de escolarização,Educação Infantil,NaN,-31.77139,-52.32994
4,ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATE...,EEI CHAPEUZINHO VERMELHO,43002145,RS,Pelotas,Urbana,A escola não está em área de localização difer...,Privada,"RUA BARAO DE SANTA TECLA, 621 CENTRO. 96010-14...",(53) 81265782,Privada,Particular,Não,Sim,Até 50 matrículas de escolarização,Educação Infantil,NaN,-31.76221,-52.34358


In [6]:
infoSchools.shape

(302, 19)

In [7]:
infoSchools.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 19 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Restrição de Atendimento                  302 non-null    object 
 1   Escola                                    302 non-null    object 
 2   Código INEP                               302 non-null    int64  
 3   UF                                        302 non-null    object 
 4   Município                                 302 non-null    object 
 5   Localização                               302 non-null    object 
 6   Localidade Diferenciada                   302 non-null    object 
 7   Categoria Administrativa                  302 non-null    object 
 8   Endereço                                  302 non-null    object 
 9   Telefone                                  292 non-null    object 
 10  Dependência Administrativa            

In [8]:
infoSchools.isnull().sum()

Restrição de Atendimento                      0
Escola                                        0
Código INEP                                   0
UF                                            0
Município                                     0
Localização                                   0
Localidade Diferenciada                       0
Categoria Administrativa                      0
Endereço                                      0
Telefone                                     10
Dependência Administrativa                    0
Categoria Escola Privada                      0
Conveniada Poder Público                      0
Regulamentação pelo Conselho de Educação     63
Porte da Escola                              63
Etapas e Modalidade de Ensino Oferecidas     66
Outras Ofertas Educacionais                 212
Latitude                                    104
Longitude                                   104
dtype: int64

### Transformação dos dados

* Processo de Geocoding (Não funcional):

In [9]:
infoSchools["Endereço"][2]

'AVENIDA DOMINGOS JOSE DE ALMEIDA, 1077 AREAL. 96085-470 Pelotas - RS.'

In [ ]:
geolocator = Nominatim(user_agent = "ScoolarData")
latitude = []
longitude = []

for local in infoSchools["Endereço"]:
    try:
        location = geolocator.geocode(local)
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        continue

* Conversão do *dataframe* para json:

In [16]:
schoolData = dataframeToJSONConverter (infoSchools)
schoolData[1]

{'Restrição de Atendimento': 'ESCOLA PARALISADA',
 'Escola': 'EEI NOSSOS AMIGUINHOS',
 'Código INEP': 43001084,
 'UF': 'RS',
 'Município': 'Pelotas',
 'Localização': 'Urbana',
 'Localidade Diferenciada': 'Não Informado',
 'Categoria Administrativa': 'Privada',
 'Endereço': 'RUA VOLUNTARIOS DA PATRIA, 1927. CENTRO. 96015-730 Pelotas - RS.',
 'Telefone': '(53) 30256230',
 'Dependência Administrativa': 'Privada',
 'Categoria Escola Privada': 'Não Informado',
 'Conveniada Poder Público': 'Não',
 'Regulamentação pelo Conselho de Educação': None,
 'Porte da Escola': None,
 'Etapas e Modalidade de Ensino Oferecidas': None,
 'Outras Ofertas Educacionais': None,
 'Latitude': None,
 'Longitude': None}

### Carregamento dos dados

* Inserindo no Banco de Dados (MongoDB)

In [13]:
collection.insert_many(schoolData)

In [14]:
collection.find_one({"Código INEP": 43000304})

{'_id': ObjectId('5f2620d181e7e7953bdb84af'),
 'Restrição de Atendimento': 'ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATENDIMENTO',
 'Escola': 'COLEGIO TIRADENTES PELOTAS',
 'Código INEP': 43000304,
 'UF': 'RS',
 'Município': 'Pelotas',
 'Localização': 'Urbana',
 'Localidade Diferenciada': 'A escola não está em área de localização diferenciada',
 'Categoria Administrativa': 'Pública',
 'Endereço': 'AV DUQUE DE CAXIAS 546, 546 FRAGATA. 96030-003 Pelotas - RS.',
 'Telefone': '(53) 32812866',
 'Dependência Administrativa': 'Estadual',
 'Categoria Escola Privada': 'Não Informado',
 'Conveniada Poder Público': 'Não',
 'Regulamentação pelo Conselho de Educação': 'Sim',
 'Porte da Escola': 'Entre 201 e 500 matrículas de escolarização',
 'Etapas e Modalidade de Ensino Oferecidas': 'Ensino Médio',
 'Outras Ofertas Educacionais': None,
 'Latitude': -31.75701,
 'Longitude': -52.37308}